### Tarea 3

In [1]:
import csv
import timeit
import datetime
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd
import os
from pyspark.sql import Window, Column
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
folder = "book/"

24/12/25 17:08:52 WARN Utils: Your hostname, Zhijie resolves to a loopback address: 127.0.1.1; using 192.168.1.58 instead (on interface wlo1)
24/12/25 17:08:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/25 17:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def filter_data(line):
    if 'User' in line:
        return False
    return True

def split_data(data):
    datas = data.split(';')
    ISBN = datas[1].strip().strip("'").strip("\\").strip('"').strip("#").strip("(").strip(")")
    return int(datas[0]), ISBN, int(datas[2])

book_fields = [
    StructField("ISBN", StringType(), False), StructField("Title", StringType(), False), StructField("Author", StringType(), False),
    StructField("Year", IntegerType(), False), StructField("Publisher", StringType(), False)
]
book_schema = StructType(book_fields)

rating_fields = [
    StructField("UserId", IntegerType(), False), StructField("ISBN", StringType(), False), StructField("Rating", IntegerType(), True)
]
rating_schema = StructType(rating_fields)

user_fields = [
    StructField("UserId", IntegerType(), False), StructField("Age", IntegerType(), True)
]
user_schema = StructType(user_fields)

if not os.path.exists(folder + "Books.parquet"):
    data = spark.read.csv(folder+"Books.csv", header=True, schema=book_schema, sep=";")
    data.write.save(folder+"Books.parquet")

if not os.path.exists(folder + "Ratings.parquet"):
    from pyspark import SparkContext
    sc = SparkContext.getOrCreate()
    data = spark.createDataFrame(sc.textFile(folder+'Ratings.csv', 8).filter(filter_data).map(split_data), rating_schema)
    data.write.save(folder+"Ratings.parquet")

if not os.path.exists(folder + "Users.parquet"):
    data = spark.read.csv(folder+"Users.csv", header=True, schema=user_schema, sep=";")
    data.write.save(folder+"Users.parquet")


In [3]:
books = spark.read.load(folder+"Books.parquet").distinct()
ratings = spark.read.load(folder+"Ratings.parquet").distinct()
users = spark.read.load(folder+"Users.parquet").distinct().fillna({"age": 0})

join = users.join(ratings, 'UserId', 'inner')
data = join.join(books, 'ISBN', 'left')

print("Books:", books.count())
print("Ratings:", ratings.count())
print("Users:", users.count())
print("Joined:", join.count())
print("Joined:", data.count())

Books: 271378


Ratings: 1149780
Users: 278859


Joined: 1149780
Joined: 1149780


24/12/25 17:09:12 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Se ha hecho una prueba rápida para ver si los valores tras realizar el join tienen sentido. Además, se ha observado que hay bastantes valores que no pueden realizar un inner join ya que falta datos en alguna de las tablas o porque existe valores que estaban mal puestas. Se ha intentado corregir alguna de ellas pero no todas.

In [4]:
data.show(10)

+----------+------+---+------+--------------------+--------------------+----+--------------------+
|      ISBN|UserId|Age|Rating|               Title|              Author|Year|           Publisher|
+----------+------+---+------+--------------------+--------------------+----+--------------------+
|0312966970|276957|  0|     6|Four To Score (A ...|     Janet Evanovich|1999|St. Martin's Pape...|
|0316168688|277427| 48|     0|The Dogs of Babel...|   Carolyn Parkhurst|2003|       Little, Brown|
|0441005470|277157| 70|     0|         Mockingbird|        Sean Stewart|1998|           Ace Books|
|0449221873|277639| 48|     0|              Mexico|   James A. Michener|1994|       Fawcett Books|
|055358068X|277139|  0|     0|The Least Likely ...|        Jane Feather|2000|              Bantam|
|0679746048|277195| 54|     0|   Girl, Interrupted|      SUSANNA KAYSEN|1994|             Vintage|
|0722539924|277231|  0|     7|                NULL|                NULL|NULL|                NULL|
|157174056

In [6]:
data.filter("author IS NULL").show(10)

+-------------+------+---+------+-----+------+----+---------+
|         ISBN|UserId|Age|Rating|Title|Author|Year|Publisher|
+-------------+------+---+------+-----+------+----+---------+
|0 440 20615 4| 71726| 18|     9| NULL|  NULL|NULL|     NULL|
|0 7525 1962 x|266641|  0|     0| NULL|  NULL|NULL|     NULL|
|0.380.44099.7| 20919|  0|    10| NULL|  NULL|NULL|     NULL|
| 000000046565|159014|  0|     1| NULL|  NULL|NULL|     NULL|
| 000001246971|213324| 48|     0| NULL|  NULL|NULL|     NULL|
| 000001246971|268423| 42|     8| NULL|  NULL|NULL|     NULL|
| 000001246971| 11776| 31|     8| NULL|  NULL|NULL|     NULL|
|  0000225820X|242060| 50|     5| NULL|  NULL|NULL|     NULL|
|  00006715885| 33759|  0|     0| NULL|  NULL|NULL|     NULL|
|  00006715885|158856|  0|     0| NULL|  NULL|NULL|     NULL|
+-------------+------+---+------+-----+------+----+---------+
only showing top 10 rows



Uno de los ejemplos sería este, donde existe ISBN que está totalmente mal, pero como desconocemos el formato correcto, no podemos hacer nada.

### Lista de usuarios junto con el número de libros que han valorado

In [7]:
data = users.join(ratings, 'UserId', 'left')
data.groupBy("UserId").agg(F.count(F.when(data.ISBN.isNotNull(), 1)).alias("BookCount")).orderBy("BookCount", ascending=False).collect()

[Row(UserId=11676, BookCount=13602),
 Row(UserId=198711, BookCount=7550),
 Row(UserId=153662, BookCount=6109),
 Row(UserId=98391, BookCount=5891),
 Row(UserId=35859, BookCount=5850),
 Row(UserId=212898, BookCount=4785),
 Row(UserId=278418, BookCount=4533),
 Row(UserId=76352, BookCount=3367),
 Row(UserId=110973, BookCount=3100),
 Row(UserId=235105, BookCount=3067),
 Row(UserId=230522, BookCount=2991),
 Row(UserId=16795, BookCount=2948),
 Row(UserId=234623, BookCount=2674),
 Row(UserId=36836, BookCount=2529),
 Row(UserId=52584, BookCount=2512),
 Row(UserId=245963, BookCount=2507),
 Row(UserId=204864, BookCount=2504),
 Row(UserId=55492, BookCount=2459),
 Row(UserId=185233, BookCount=2448),
 Row(UserId=171118, BookCount=2421),
 Row(UserId=102967, BookCount=2352),
 Row(UserId=232131, BookCount=2347),
 Row(UserId=227447, BookCount=2340),
 Row(UserId=129358, BookCount=2317),
 Row(UserId=98741, BookCount=2317),
 Row(UserId=60244, BookCount=2236),
 Row(UserId=190925, BookCount=2154),
 Row(UserI

Para listar usuarios junto con el número de libreos que han valorado, sólo tenemos que realizar un join entre User y Rating, utilizando left join. El motivo no realizar join con los libros es que en la tabla de Rating, ya tenemos el ISBN de los libros que han valorado, por lo que realizar un join extra sería perder el tiempo. Y el motivo de realizar left join es porque habrá usuarios que no han valorado ningún libro y nos enteresa saber esa información.

Finalmente, se utiliza la función F.when(data.ISBN.isNotNull(), 1) para contar los libros leidos y ordenamos el conteo por orden descendente.

#### Rating máximo recibido por cada editorial

In [8]:
data = books.join(ratings, 'ISBN', 'left')
data.groupBy("publisher").agg(F.max(data.Rating)).collect()

[Row(publisher='Harper Mass Market Paperbacks (Mm)', max(Rating)=10),
 Row(publisher='LPC Group', max(Rating)=9),
 Row(publisher='Chicago Review Press', max(Rating)=10),
 Row(publisher='Adams Media Corp', max(Rating)=10),
 Row(publisher='Celestial Arts', max(Rating)=10),
 Row(publisher='Cleis Press', max(Rating)=10),
 Row(publisher='Chosen Books', max(Rating)=10),
 Row(publisher='Carroll &amp; Graf Publishers', max(Rating)=10),
 Row(publisher='Aqua Explorers', max(Rating)=9),
 Row(publisher='Houghton Mifflin Co (Jp)', max(Rating)=10),
 Row(publisher='Ramana Pubn', max(Rating)=0),
 Row(publisher='Regina Press Malhame & Company', max(Rating)=9),
 Row(publisher='Joshua Odell Editions Capra Press', max(Rating)=10),
 Row(publisher='No Exit Press', max(Rating)=7),
 Row(publisher='Lorenz Books', max(Rating)=10),
 Row(publisher='High Country Publishers', max(Rating)=10),
 Row(publisher='Ediciones Alfaguara, S.A.', max(Rating)=10),
 Row(publisher='Ullstein-Taschenbuch-Verlag, Zweigniederlassung

Para este problema, solamente haría falta hacer join entre Books y Ratings, también con Left Join, ya que los usuarios en principio no pinta nada en este caso.
Como es el Rating máximo recibido por cada editorial, sólamente haría falta agrupar por publisher y hacer un max de rating.

#### Nombre del autor que ha recibido más ratings

In [72]:
data = books.join(ratings, 'ISBN', 'left')
data.groupBy("Author").agg(F.count(data.Rating).alias("NumRatings")).orderBy("NumRatings", ascending=False).select("Author").first()

Row(Author='Stephen King')

Para saber el nombre del author que ha recibido más rating, primero hay que hacer un join entre Books y Ratings, luego se agrupa por author para tener una lista de ratings que ha recibido, con esa información, pordríamos contar el número de rating, y ordenar por orden descendente sacando el primer valor.

#### ¿Cuál es el título del libro con mayor número de ratings para cada editorial?

In [16]:
data = books.join(ratings, 'ISBN', 'left')
overRating = Window.partitionBy(data.Publisher).orderBy(F.desc("num_ratings"))
# data.groupBy("Publisher", "Title").agg(F.count(data.Rating).alias("num_ratings")).select("Publisher", "Title", "num_ratings").collect()
data.groupBy("Publisher", "Title").agg(F.count(data.Rating).alias("num_ratings")).withColumn("rank", F.dense_rank().over(overRating)).where("rank <= 1").select("Publisher", "Title").collect()

[Row(Publisher='"""Otokar Kersovani\\"""', Title='"Gdje pijevac ne pjeva (Biblioteka ""Zlatni paun\\"")"'),
 Row(Publisher='"Flemish-Netherlands Foundation ""Stichting Ons Erfdeel,\\"""', Title='Contemporary poetry of the low countries'),
 Row(Publisher='(3 Queen Sq., WC1N 3AU), Faber and Faber Ltd', Title='The big chapel'),
 Row(Publisher='101 Productions; [distributed by Scribner, New York]', Title='Manna: foods of the frontier'),
 Row(Publisher='22nd. Century, New York', Title='Portraits of Cities'),
 Row(Publisher='3', Title="Restoring Intimacy: The Patient's Guide to Maintaining Relationships During Depression"),
 Row(Publisher='300Incredible.com', Title='300 Incredible Things for Sports Fans on the Internet'),
 Row(Publisher='3D Press', Title='Denver Hiking Guide:  45 Hikes within 45 Minutes of Denver'),
 Row(Publisher='3H Productions, Inc.', Title="Haley's Cleaning Hints"),
 Row(Publisher='3rd Bed', Title='Stories in the Worst Way'),
 Row(Publisher='3rd Woman Press', Title='Dict

Este problema es muy parecido al anterior pero con una pequeña diferencia, que es agrupar por editorial y por título, para poder contar el número de ratings que ha recibido cada publisher. Para ello, se agrupa los datos y se crea un Window Function para poder ordenar por número de ratings y sacar los valores que están en ranking 1.

#### ¿Cuál es la diferencia entre el número de ratings de cada libro y el número de ratings del libro con mayor número de ratings de la misma editorial?

In [23]:
data = books.join(ratings, 'ISBN', 'left')
overRating = Window.partitionBy(data.Publisher).orderBy(F.desc("num_ratings"))
data.groupBy("Publisher", "Title").agg(F.count(data.Rating).alias("num_ratings")).withColumn("diff", F.max("num_ratings").over(overRating) - F.col("num_ratings")).select("Publisher", "Title", "diff").collect()

[Row(Publisher='"""Otokar Kersovani\\"""', Title='"Gdje pijevac ne pjeva (Biblioteka ""Zlatni paun\\"")"', diff=0),
 Row(Publisher='"Flemish-Netherlands Foundation ""Stichting Ons Erfdeel,\\"""', Title='Contemporary poetry of the low countries', diff=0),
 Row(Publisher='3D Press', Title='Denver Hiking Guide:  45 Hikes within 45 Minutes of Denver', diff=0),
 Row(Publisher='3H Productions, Inc.', Title="Haley's Cleaning Hints", diff=0),
 Row(Publisher='A H M Publications', Title='The Subjection of Women (Crofts Classics)', diff=0),
 Row(Publisher='A. & C. Black', Title="Writing for television in the 70's", diff=0),
 Row(Publisher='A. & M. Muchnik', Title='Comuna Verdad (Analectas)', diff=0),
 Row(Publisher='A. Deutsch', Title='Moon Tiger', diff=0),
 Row(Publisher='A. Deutsch', Title='Russia in revolution, 1900-1930', diff=1),
 Row(Publisher='A. Deutsch', Title='Apple of my eye', diff=1),
 Row(Publisher='A. Deutsch', Title='Pictures from the water trade: An Englishman in Japan', diff=1),


De nuevo, este problema es parecido justo al problema anterior pero obteniendo la diferencia de rating. Por lo tanto, la partición que se crea es la misma pero a la hora de aplicar la función de Window, se debería hacer una resta del max número de ratings de esa editorial con el número ratings del libro actual.